In [ ]:
!pip install -q accelerate==0.21.0 bitsandbytes==0.40.2 transformers sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 60.3 MB/s eta 0:00:00


In [ ]:
!pip install datasets faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.8 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed

In [ ]:

from huggingface_hub import login
import os
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline,
    logging,
)

# Se connecter à Hugging Face avec le token
# Hugging Face login token
huggingface_token = "HUGGINGFACE_TOKEN"

# Se connecter à Hugging Face avec le token
login(token=huggingface_token)
use_4bit = True
compute_dtype = getattr(torch, "float16")
device_map = {"": 0}

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
#meta-llama/Meta-Llama-3.1-8B-Instruct
#meta-llama/Llama-2-7b-chat-hf
# Load base model
print(model_id)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    quantization_config=bnb_config
)
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful
meta-llama/Meta-Llama-3-8B-Instruct


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

You are calling `save_pretrained` to a 4-bit converted model, but your `bitsandbytes` version doesn't support it. If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed.


In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from datasets import Dataset, DatasetDict
ST = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
#la fonction pour générer les embeddings

df = pd.read_excel("dataset_agriculture_18K.xlsx")
def nettoyer_texte(texte):
    texte = texte.replace('#', '')
    texte = texte.replace('\xa0', ' ')
    texte = texte.replace('\n', ' ')
    texte = texte.replace('**', '')
    texte = texte.strip()
    return texte
df=df.dropna()

df['Prompt_context'] = df['Prompt_context'].apply(lambda x: nettoyer_texte(str(x)))



def embed(batch):
    information = batch['Prompt_context']
    return {
        "embeddings": ST.encode(information, show_progress_bar=False, batch_size=1, device='cpu')
    }
dataset = Dataset.from_pandas(df)
# Appliquez la fonction d'intégration aux données
dataset = dataset.map(embed, batched=True, batch_size=16)
data = dataset.add_faiss_index("embeddings")

Map:   0%|          | 0/13115 [00:00<?, ? examples/s]

  0%|          | 0/14 [00:00<?, ?it/s]

In [ ]:
# column name that has the embeddings of the dataset

def search(query: str ):

    """a function that embeds a new query and returns the most probable results"""
    embedded_query = ST.encode(query) # embed new query
    scores, retrieved_examples = data.get_nearest_examples( # retrieve results
        "embeddings", embedded_query, # compare our new embedded query with the dataset embeddings
    )
    return scores, retrieved_examples


In [ ]:
def search(query: str ):
    embedded_query = ST.encode(query) # embed new query
    scores, retrieved_examples = data.get_nearest_examples( # retrieve results
        "embeddings", embedded_query, # compare new embedded query with the dataset embeddings
    )
    return scores, retrieved_examples
SYS_PROMPT = """Vous êtes un assistant chargé de répondre aux questions des agriculteurs marocains.
On vous donne des extraits de documents ainsi qu'une question. Fournissez une réponse conversationnelle et précise.
La réponse doit être dans la même langue que la question posée par l'utilisateur.
Si les documents fournis ne sont pas pertinents pour la question, ne les utilisez pas."""
def format_prompt(prompt,retrieved_documents,k):
    PROMPT = f"Question:{prompt}\nContext:"
    for idx in range(k) :
        PROMPT+= f"{retrieved_documents['Prompt_context'][idx]}\n"
    return PROMPT
def generate(formatted_prompt):
    formatted_prompt = formatted_prompt[:2000] # to avoid GPU OOM
    messages = [{"role":"system","content":SYS_PROMPT},{"role":"user","content":formatted_prompt}]
    input_ids = tokenizer.apply_chat_template(
      messages,
      add_generation_prompt=True,
      return_tensors="pt"
      ).to(model.device)
    outputs = model.generate(
      input_ids,
      max_new_tokens=1024,
      eos_token_id=terminators,
      do_sample=True,
      temperature=0.4,
      top_k=50,
      top_p=0.95,
      repetition_penalty=1.15
  )
    response = outputs[0][input_ids.shape[-1]:]
    return tokenizer.decode(response, skip_special_tokens=True)

def rag_chatbot(prompt:str,k:int=3):
    scores, retrieved_documents = search(prompt)
    print(retrieved_documents)
    formatted_prompt = format_prompt(prompt, retrieved_documents, k)
    response = generate(formatted_prompt)
    return response





